<a href="https://colab.research.google.com/github/KrischML/cmi-kaggle-comp/blob/main/001_cmi_note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

cmi_detect_behavior_with_sensor_data_path = kagglehub.competition_download('cmi-detect-behavior-with-sensor-data')

print('Data source import complete.')

In [ ]:
import os
import pandas as pd
import polars as pl
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
for root, dirs, files in os.walk('/'):
    for file in files:
        if file.endswith('.csv'):
            print(os.path.join(root, file))

In [ ]:
/root/.cache/kagglehub/competitions/cmi-detect-behavior-with-sensor-data/train.csv
/root/.cache/kagglehub/competitions/cmi-detect-behavior-with-sensor-data/test_demographics.csv
/root/.cache/kagglehub/competitions/cmi-detect-behavior-with-sensor-data/test.csv
/root/.cache/kagglehub/competitions/cmi-detect-behavior-with-sensor-data/train_demographics.csv

In [ ]:
class CFG:

    ip_path: str = "/root/.cache/kagglehub/competitions/cmi-detect-behavior-with-sensor-data/"

    random_state: int = 42

    LE = LabelEncoder()


print(CFG.ip_path)

In [ ]:
train_df = pd.read_csv(CFG.ip_path + "train.csv")
train_demographics_df  = pd.read_csv(CFG.ip_path + "train_demographics.csv")

test_df = pd.read_csv(CFG.ip_path + "test.csv")
test_demographics_df = pd.read_csv(CFG.ip_path + "test_demographics.csv")

In [ ]:
# Merge demographic information into the main training dataframe
train_demo_df = train_df.merge(train_demographics_df, on="subject", how="left")



In [ ]:
train_demo_df.head(2)

In [ ]:
# Annahme: train_df enthält die Spalten
df = train_demo_df.copy()

df['sequence_id_encoded'] = CFG.LE.fit_transform(df['sequence_id'])


corr_df = df[['sequence_id_encoded', 'sequence_counter']]

# Korrelation berechnen
corr = corr_df.corr()

# Heatmap plotten
plt.figure(figsize=(6, 5))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, fmt=".2f")
plt.title("Korrelation: sequence_id vs. sequence_counter")
plt.tight_layout()
plt.show()


In [ ]:
train_demo_df['sequence_id'].unique()


In [ ]:

acc_sencor_cols = [col for col in train_df.columns if any(s in col for s in ['acc_'])]
rot_sensor_cols = [col for col in train_df.columns if any(s in col for s in ['rot_'])]
thm_sensor_cols = [col for col in train_df.columns if any(s in col for s in ['thm_'])]
tof_sensor_cols = [col for col in train_df.columns if any(s in col for s in ['tof_'])]

thm_sensor_cols

In [ ]:


corr_df = train_demo_df[thm_sensor_cols]
# Korrelation berechnen
corr = corr_df.corr()

# Heatmap plotten
plt.figure(figsize=(6, 5))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, fmt=".2f")
plt.title("Korrelation: sequence_id vs. sequence_counter")
plt.tight_layout()
plt.show()

In [ ]:
all_sensor_cols = [col for col in train_df.columns if any(s in col for s in ['acc_', 'rot_', 'thm_', 'tof_'])]
all_sensor_cols

In [ ]:
train_demo_df.head(2)

In [ ]:
cat_columns = [col for col in train_demo_df.columns if train_demo_df[col].dtype == 'object']
cat_columns

In [ ]:
train_demo_df['thm_5'].isnull().sort_values(ascending=False)

In [ ]:
# Encode gesture labels
le = LabelEncoder()
train['gesture_encoded'] = le.fit_transform(train['gesture'])

# Features and target
features = ['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']
X = train[features]
y = train['gesture_encoded']